# Selenium Webscraping
## Notebook de clase

En este notebook podras encontrar el ejercicio que hicimos en clase para aprender las bases de Selenium.

Adicionalmente, se puede consultar la documentación oficial: https://www.selenium.dev/documentation/webdriver/

Hagamos los imports relevantes y conectémonos al driver de Chrome que está montado en docker. 

In [ ]:
!docker stop sel-docker
!docker rm sel-docker
!docker run -d --name "sel-docker" -p 4444:4444 --shm-size=2g \
  -e SE_NODE_MAX_SESSIONS=6 \
  -e SE_NODE_SESSION_TIMEOUT=1200 \ 
  -e SE_VNC_NO_PASSWORD=1 \
  selenium/standalone-chrome

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Remote(command_executor='http://localhost:4444/wd/hub',options = webdriver.ChromeOptions())

El siguiente código solo lo vamos a usar en caso de tener el Chrome Driver local.

In [9]:
#Por si falla algo. URL al driver de Chrome (descargar el driver desde https://sites.google.com/chromium.org/driver/)
#driver = webdriver.Chrome(options = webdriver.ChromeOptions())

Utilizaremos un sitio web de aprendizaje. Nota: en la vida diaria los sitios se van actualizando, lo cual puede dificultar mantener el código al día y funcionando 

In [10]:
url = 'https://scrapepark.org/courses/spanish/'

# Abrir la página web
driver.get(url)

No es necesario que se abra la ventana de Google, pero al principio es muy útil para saber qué está pasando exactamente con nuestra automatización. Una vez que terminamos de debuggear, es recomendable utilizar el driver sin visualizar la ventana para ahorrar RAM y hacer más rápido los procesos.

In [5]:
#options = webdriver.ChromeOptions()
#options.add_argument('--headless')  # Ejecutar en modo headless (sin interfaz gráfica)


In [4]:
#driver.close()

In [34]:
driver = webdriver.Remote(command_executor='http://localhost:4444/wd/hub',options = webdriver.ChromeOptions())
#driver = webdriver.Chrome(options = options)

En este caso es importante cerrar las sesiones con el siguiente comando:

In [35]:
#driver.close()

## **Scrapeo básico** 

 ``driver.find_element(By.<>,'html')``


In [11]:
nom_pagina = driver.title #titulo de la página
print("El título de la página es:", nom_pagina)


El título de la página es: ScrapePark.org


Hora de meternos al mundo de la inspección:

In [13]:
#Obtener la información de la página hallada en el footer e imprimir su contenido:
info_marca = driver.find_element(By.CLASS_NAME,"information-f")
print(info_marca.text)

DIRECCIÓN: Calle falsa 123
TELÉFONO: 4-444-4444
EMAIL: ventas@mail.com


In [18]:
testimonio= driver.find_element(By.ID,'testimonios')
print(testimonio.text)

Testimonios de clientes
Cliente 3
Estoy muy conforme. Hay muchas patinetas y los diseños son fantásticos.
Anterior
Siguiente


In [22]:
#Obtener los testimonios:
import time

testimonio= driver.find_element(By.ID,'testimonios')
print(testimonio.text)

time.sleep(.5)

testimonio= driver.find_element(By.ID,'testimonios')
print(testimonio.text)


Testimonios de clientes
Cliente 3
Estoy muy conforme. Hay muchas patinetas y los diseños son fantásticos.
Anterior
Siguiente
Testimonios de clientes
Cliente 3
Estoy muy conforme. Hay muchas patinetas y los diseños son fantásticos.
Anterior
Siguiente


¿Qué pasa si queremos el testimonio de todos los clientes? Necesitamos interectuar con el sitio.

``.click()``

In [ ]:
#Interactuar con el sitio haciendo click() para obtener distintos testimonios:

¿Qué no estamos tomando en cuenta?

In [ ]:
#Arreglar el código de arriba

**¿Cómo localizar un objeto usando su Xpath.**

Ventaja: efectividad; 
Desventaja: trabajo

Xpath Syntax:
//tagName[@AttributeName="Value"]

Obtengamos los productos de la página

In [25]:

botonProductos = driver.find_element(By.XPATH,'//*[@id="navbarSupportedContent"]/ul/li[4]/a')

botonProductos.click()

¿Y si la página cambia según el tamaño de la ventana?

In [28]:
detailBoxes =driver.find_elements(By.XPATH,'//div[@class= "detail-box"]')

for box in detailBoxes:
    print(box.text)
    print("")



Descuentos 20% Off
¡En todos los productos!
Aprovechá nuestras ofertas.
Comprar ahora



Entrega rápida

Envío gratuito

La mejor calidad

Patineta Nueva 1
$75

Patineta Usada 2
$80

Patineta Nueva 3
$68

Patineta Usada 4
$70

Patineta Nueva 5
$75

Patineta Nueva 6
$58

Patineta Nueva 7
$80

Patineta Nueva 8
$35

Patineta Nueva 9
$165

Patineta Usada 10
$54

Patineta Usada 11
$99

Patineta Nueva 12
$110



Cliente 2
¡La calidad y variedad de patinetas es impresionante! Definitivamente volveré a comprar.





Regex para formatear los datos scrapeados

In [29]:
#Queremos crear un data frame de los productos con sus precios 
#usaremos regex para obtener solo el texto de las boxes que necesitamos
import re

patronProducto= r"Patineta (Nueva|Usada) \d{1,2}"

matches = [box for box in detailBoxes if re.search(patronProducto,box.text)]

for box in matches:
    print(box.text)

Patineta Nueva 1
$75
Patineta Usada 2
$80
Patineta Nueva 3
$68
Patineta Usada 4
$70
Patineta Nueva 5
$75
Patineta Nueva 6
$58
Patineta Nueva 7
$80
Patineta Nueva 8
$35
Patineta Nueva 9
$165
Patineta Usada 10
$54
Patineta Usada 11
$99
Patineta Nueva 12
$110


In [30]:
import pandas as pd

pattern = r"Patineta (Nueva|Usada) (\d{1,2})\s*\n*\s*\$(\d+)" #ahora más específico

# Crear listas para almacenar los productos, estados, números y precios
productos = [] #un producto es el estado de la patineta con su número
estados = []
numeros = []
precios = []

# Iteramos
for match in matches:
    resultado = re.search(pattern, match.text)
    if resultado:
        #separamos y definimos variables
        tipo = resultado.group(1) + (resultado.group(2))
        estado = resultado.group(1) 
        numero = int(resultado.group(2))
        precio = int(resultado.group(3))
        
        #agregamos a las listas
        productos.append(tipo)
        estados.append(estado)
        numeros.append(numero)
        precios.append(precio)

# Crear el DataFrame usando pandas
data = {
    'Producto': productos,
    'Estado': estados,
    'Número': numeros,
    'Precio': precios
}

df = pd.DataFrame(data)

df

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,Producto,Estado,Número,Precio
0,Nueva1,Nueva,1,75
1,Usada2,Usada,2,80
2,Nueva3,Nueva,3,68
3,Usada4,Usada,4,70
4,Nueva5,Nueva,5,75
5,Nueva6,Nueva,6,58
6,Nueva7,Nueva,7,80
7,Nueva8,Nueva,8,35
8,Nueva9,Nueva,9,165
9,Usada10,Usada,10,54


Ya treniendo el data frame podemos usar las funciones de pandas para analizar los datos

In [31]:
print('Precio promedio:',df['Precio'].mean())
print('Precio mínimo:',df['Precio'].min())
print('Precio máximo',df['Precio'].max())

Precio promedio: 80.75
Precio mínimo: 35
Precio máximo 165


In [32]:
print('Precio promedio de las patinetas usadas', df[df['Estado'] == 'Usada']['Precio'].mean())
print('Precio promedio de las patinetas nuevas', df[df['Estado'] == 'Nueva']['Precio'].mean())


print('Rango de precios de las patinetas usadas', df[df['Estado'] == 'Usada']['Precio'].max()-df[df['Estado'] == 'Usada']['Precio'].min())
print('Rango de precios de las patinetas nuevas', df[df['Estado'] == 'Nueva']['Precio'].max()-df[df['Estado'] == 'Nueva']['Precio'].min())

Precio promedio de las patinetas usadas 75.75
Precio promedio de las patinetas nuevas 83.25
Rango de precios de las patinetas usadas 45
Rango de precios de las patinetas nuevas 130


In [33]:
driver.quit()

### ¡Acabamos! ###

In [ ]:
!docker stop sel-docker
!docker rm sel-docker